### Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import preprocessing

In [2]:
df = pd.read_parquet("Trades_Quote.parquet")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991986 entries, 0 to 1991985
Data columns (total 11 columns):
 #   Column      Dtype         
---  ------      -----         
 0   COMPANY     object        
 1   DATE        object        
 2   TRADE_TIME  datetime64[us]
 3   QUOTE_TIME  datetime64[us]
 4   EX          object        
 5   SIZE        int64         
 6   PRICE       float64       
 7   BID         float64       
 8   BIDSIZ      float64       
 9   ASK         float64       
 10  ASKSIZ      float64       
dtypes: datetime64[us](2), float64(5), int64(1), object(3)
memory usage: 167.2+ MB


,COMPANY,DATE,TRADE_TIME,QUOTE_TIME,EX,SIZE,PRICE,BID,BIDSIZ,ASK,ASKSIZ
0,C,2024-01-02,2024-01-02 09:32:40.845895,2024-01-02 09:32:40.842364,H,100,51.33,51.33,1.0,51.36,1.0
1,C,2024-01-02,2024-01-02 09:32:40.888035,2024-01-02 09:32:40.888012,H,100,51.33,51.32,1.0,51.35,1.0
2,C,2024-01-02,2024-01-02 09:32:40.888311,2024-01-02 09:32:40.888308,H,50,51.33,51.32,1.0,51.35,1.0
3,C,2024-01-02,2024-01-02 09:32:40.929882,2024-01-02 09:32:40.897981,H,200,51.31,51.31,2.0,51.33,1.0
4,C,2024-01-02,2024-01-02 09:32:40.953091,2024-01-02 09:32:40.937756,H,1,51.30,51.30,3.0,51.32,1.0


In [3]:
# Create 1-minute interval
df['INTERVAL_TIME'] = df['TRADE_TIME'].dt.floor('min')
df.head()

,COMPANY,DATE,TRADE_TIME,QUOTE_TIME,EX,SIZE,PRICE,BID,BIDSIZ,ASK,ASKSIZ,INTERVAL_TIME
0,C,2024-01-02,2024-01-02 09:32:40.845895,2024-01-02 09:32:40.842364,H,100,51.33,51.33,1.0,51.36,1.0,2024-01-02 09:32:00
1,C,2024-01-02,2024-01-02 09:32:40.888035,2024-01-02 09:32:40.888012,H,100,51.33,51.32,1.0,51.35,1.0,2024-01-02 09:32:00
2,C,2024-01-02,2024-01-02 09:32:40.888311,2024-01-02 09:32:40.888308,H,50,51.33,51.32,1.0,51.35,1.0,2024-01-02 09:32:00
3,C,2024-01-02,2024-01-02 09:32:40.929882,2024-01-02 09:32:40.897981,H,200,51.31,51.31,2.0,51.33,1.0,2024-01-02 09:32:00
4,C,2024-01-02,2024-01-02 09:32:40.953091,2024-01-02 09:32:40.937756,H,1,51.30,51.30,3.0,51.32,1.0,2024-01-02 09:32:00


In [4]:
# Calculate Variables: last midprice, quoted spread, relative quoted spread, effective relative spread, order imbalance
# and depth imbalance, number of trades and total traded volume [+1-minute returns and relaized volatility over the past hour]

df['MID_PRICE'] = (df['BID'] + df['ASK']) / 2.0
df['QUOTED_SPREAD'] = df['ASK'] - df['BID']
df['RELATIVE_SPREAD'] = (df['QUOTED_SPREAD'] / df['MID_PRICE'].replace(0, np.nan))
df['EFFECTIVE_RELATIVE_SPREAD'] = (2 * np.abs((df['PRICE'] - df['MID_PRICE']) / df['MID_PRICE'].replace(0, np.nan)))
df['DEPTH_IMBALANCE'] = ((df['ASKSIZ'] - df['BIDSIZ']) / (df['BIDSIZ'] + df['ASKSIZ']).replace(0, np.nan))

# Classify trades as buyer-initiated or seller-initiated
df['BUY_VOLUME'] = np.where(df['PRICE'] >= df['ASK'], df['SIZE'], 0)
df['SELL_VOLUME'] = np.where(df['PRICE'] <= df['BID'], df['SIZE'], 0)

df.head()

,COMPANY,DATE,TRADE_TIME,QUOTE_TIME,EX,SIZE,PRICE,BID,BIDSIZ,ASK,ASKSIZ,INTERVAL_TIME,MID_PRICE,QUOTED_SPREAD,RELATIVE_SPREAD,EFFECTIVE_RELATIVE_SPREAD,DEPTH_IMBALANCE,BUY_VOLUME,SELL_VOLUME
0,C,2024-01-02,2024-01-02 09:32:40.845895,2024-01-02 09:32:40.842364,H,100,51.33,51.33,1.0,51.36,1.0,2024-01-02 09:32:00,51.345,0.03,0.000584,0.000584,0.000000,0,100
1,C,2024-01-02,2024-01-02 09:32:40.888035,2024-01-02 09:32:40.888012,H,100,51.33,51.32,1.0,51.35,1.0,2024-01-02 09:32:00,51.335,0.03,0.000584,0.000195,0.000000,0,0
2,C,2024-01-02,2024-01-02 09:32:40.888311,2024-01-02 09:32:40.888308,H,50,51.33,51.32,1.0,51.35,1.0,2024-01-02 09:32:00,51.335,0.03,0.000584,0.000195,0.000000,0,0
3,C,2024-01-02,2024-01-02 09:32:40.929882,2024-01-02 09:32:40.897981,H,200,51.31,51.31,2.0,51.33,1.0,2024-01-02 09:32:00,51.320,0.02,0.000390,0.000390,-0.333333,0,200
4,C,2024-01-02,2024-01-02 09:32:40.953091,2024-01-02 09:32:40.937756,H,1,51.30,51.30,3.0,51.32,1.0,2024-01-02 09:32:00,51.310,0.02,0.000390,0.000390,-0.500000,0,1


In [5]:
tradesquotes = df.groupby(['COMPANY', 'DATE', 'INTERVAL_TIME']).agg(
    # Price and Spread Measures
    QUOTED_SPREAD_AVG=('QUOTED_SPREAD', 'mean'),
    RELATIVE_SPREAD_AVG=('RELATIVE_SPREAD', 'mean'),
    EFFECTIVE_RELATIVE_SPREAD_AVG=('EFFECTIVE_RELATIVE_SPREAD', 'mean'),

    # Last Mid-price for each 1-minute interval
    LAST_MID_PRICE=('MID_PRICE', 'last'),

    # Size / Depth Measures
    BID_SIZE_AVG=('BIDSIZ', 'mean'),
    ASK_SIZE_AVG=('ASKSIZ', 'mean'),
    DEPTH_IMBALANCE_AVG=('DEPTH_IMBALANCE', 'mean'),

    # Order Imbalance
    BUY_VOLUME_SUM=('BUY_VOLUME', 'sum'),
    SELL_VOLUME_SUM=('SELL_VOLUME', 'sum'),
    TOTAL_VOLUME_SUM=('SIZE', 'sum'),

    # Trade Intensity
    NUM_TRADES=('PRICE', 'count'),
    TOTAL_TRADED_VOLUME=('SIZE', 'sum')
).reset_index()

# Calculate Order Imbalance
tradesquotes['ORDER_IMBALANCE'] = ((tradesquotes['BUY_VOLUME_SUM'] - tradesquotes['SELL_VOLUME_SUM']) /
                              tradesquotes['TOTAL_VOLUME_SUM'].replace(0, np.nan))

# Sort by interval_time
tradesquotes = tradesquotes.sort_values(by=['COMPANY', 'DATE', 'INTERVAL_TIME']).reset_index(drop=True)

tradesquotes.head()

,COMPANY,DATE,INTERVAL_TIME,QUOTED_SPREAD_AVG,RELATIVE_SPREAD_AVG,EFFECTIVE_RELATIVE_SPREAD_AVG,LAST_MID_PRICE,BID_SIZE_AVG,ASK_SIZE_AVG,DEPTH_IMBALANCE_AVG,BUY_VOLUME_SUM,SELL_VOLUME_SUM,TOTAL_VOLUME_SUM,NUM_TRADES,TOTAL_TRADED_VOLUME,ORDER_IMBALANCE
0,C,2024-01-02,2024-01-02 09:30:00,0.036966,0.000721,0.000293,51.280,2.744828,3.200000,0.017096,3154,4669,41191,302,41191,-0.036780
1,C,2024-01-02,2024-01-02 09:31:00,0.042653,0.000833,0.000367,51.335,2.224490,2.546939,0.063801,3333,4003,20013,248,20013,-0.033478
2,C,2024-01-02,2024-01-02 09:32:00,0.067746,0.001336,0.000924,51.485,2.689589,5.562560,0.044754,22172,22192,619970,1049,619970,-0.000032
3,C,2024-01-02,2024-01-02 09:33:00,0.080236,0.001581,0.002352,51.580,2.543906,2.503277,-0.041130,13154,8734,72906,763,72906,0.060626
4,C,2024-01-02,2024-01-02 09:34:00,0.087648,0.001741,0.008247,51.665,3.194444,2.557407,-0.093922,13499,3720,49435,540,49435,0.197815


In [6]:
#Calculate returns over previous 1-minute interval (for realized volatility) and over next 1-minute interval (as target/dependent variable)

# Sort by interval_time
tradesquotes = tradesquotes.sort_values(by=['COMPANY', 'DATE', 'INTERVAL_TIME']).reset_index(drop=True)

# Check that last time interval is actually 1 minute apart
tradesquotes['LAST_INTERVAL_TIME'] = tradesquotes.groupby(['COMPANY', 'DATE'])['INTERVAL_TIME'].shift(+1)
tradesquotes['TIME_DIFF'] = (tradesquotes['LAST_INTERVAL_TIME'] - tradesquotes['INTERVAL_TIME']).dt.total_seconds() / 60

#Count how many observations with time gap> 1 minute
count = (tradesquotes['TIME_DIFF'] > 1.5).sum()
print(f"Number of observations set to 0: {count}")

# Calculate 1-min return over the previous 1-minute interval
tradesquotes['1MIN_RETURN'] = tradesquotes.groupby(['COMPANY', 'DATE'])['LAST_MID_PRICE'].pct_change()

# Set return to 0 if time gap is not 1 minute
tradesquotes.loc[tradesquotes['TIME_DIFF'] > 1, '1MIN_RETURN'] = 0

# Clean up temporary columns
tradesquotes = tradesquotes.drop(columns=['LAST_INTERVAL_TIME', 'TIME_DIFF'])

# Calculate tardet/dependent variable -  return over the next 1-minute interval.
# Set 1-minute returns to 0 if the next interval is more than 1 minute in the future

# Calculate return and check that it's actually 1 minute apart
tradesquotes['NEXT_INTERVAL_TIME'] = tradesquotes.groupby(['COMPANY', 'DATE'])['INTERVAL_TIME'].shift(-1)
tradesquotes['TIME_DIFF'] = (tradesquotes['NEXT_INTERVAL_TIME'] - tradesquotes['INTERVAL_TIME']).dt.total_seconds() / 60

# Calculate 1-minute return over next minute
tradesquotes['NEXT_1MIN_RETURN'] = tradesquotes.groupby(['COMPANY', 'DATE'])['LAST_MID_PRICE'].pct_change().shift(-1)

#Count how many observations with time gap> 1 minute

count = (tradesquotes['TIME_DIFF'] > 1.5).sum()

print(f"Number of observations set to 0: {count}")

# Set return to 0 if time gap is not 1 minute
tradesquotes.loc[tradesquotes['TIME_DIFF'] > 1, 'NEXT_1MIN_RETURN'] = 0

# Clean up temporary columns
tradesquotes = tradesquotes.drop(columns=['NEXT_INTERVAL_TIME', 'TIME_DIFF'])

tradesquotes.head()

Number of observations set to 0: 0
Number of observations set to 0: 0


,COMPANY,DATE,INTERVAL_TIME,QUOTED_SPREAD_AVG,RELATIVE_SPREAD_AVG,EFFECTIVE_RELATIVE_SPREAD_AVG,LAST_MID_PRICE,BID_SIZE_AVG,ASK_SIZE_AVG,DEPTH_IMBALANCE_AVG,BUY_VOLUME_SUM,SELL_VOLUME_SUM,TOTAL_VOLUME_SUM,NUM_TRADES,TOTAL_TRADED_VOLUME,ORDER_IMBALANCE,1MIN_RETURN,NEXT_1MIN_RETURN
0,C,2024-01-02,2024-01-02 09:30:00,0.036966,0.000721,0.000293,51.280,2.744828,3.200000,0.017096,3154,4669,41191,302,41191,-0.036780,NaN,0.001073
1,C,2024-01-02,2024-01-02 09:31:00,0.042653,0.000833,0.000367,51.335,2.224490,2.546939,0.063801,3333,4003,20013,248,20013,-0.033478,0.001073,0.002922
2,C,2024-01-02,2024-01-02 09:32:00,0.067746,0.001336,0.000924,51.485,2.689589,5.562560,0.044754,22172,22192,619970,1049,619970,-0.000032,0.002922,0.001845
3,C,2024-01-02,2024-01-02 09:33:00,0.080236,0.001581,0.002352,51.580,2.543906,2.503277,-0.041130,13154,8734,72906,763,72906,0.060626,0.001845,0.001648
4,C,2024-01-02,2024-01-02 09:34:00,0.087648,0.001741,0.008247,51.665,3.194444,2.557407,-0.093922,13499,3720,49435,540,49435,0.197815,0.001648,-0.002807


### Winsorizing main variables at 1% and 99%

In [8]:
def winsorize(dataframe, columns, lower_percentile=1, upper_percentile=99):
    """ Simple winsorization using pandas clip """
    df_winsorized = dataframe.copy()

    for col in columns:
        if col in df.columns:
            lower = df[col].quantile(lower_percentile / 100)
            upper = df[col].quantile(upper_percentile / 100)
            df_winsorized[col] = df[col].clip(lower, upper)

    return df_winsorized

variables_to_winsorize = [
    'RELATIVE_SPREAD_AVG',
    'EFFECTIVE_RELATIVE_SPREAD_AVG',
    'QUOTED_SPREAD_AVG',
    'LAST_MID_PRICE',
    'DEPTH_IMBALANCE_AVG',
    'ORDER_IMBALANCE',
    'TOTAL_TRADED_VOLUME',
    'NUM_TRADES',
    '1MIN_RETURN',
    'NEXT_1MIN_RETURN',
]

workingdata = winsorize(tradesquotes, variables_to_winsorize)
workingdata.head()

,COMPANY,DATE,INTERVAL_TIME,QUOTED_SPREAD_AVG,RELATIVE_SPREAD_AVG,EFFECTIVE_RELATIVE_SPREAD_AVG,LAST_MID_PRICE,BID_SIZE_AVG,ASK_SIZE_AVG,DEPTH_IMBALANCE_AVG,BUY_VOLUME_SUM,SELL_VOLUME_SUM,TOTAL_VOLUME_SUM,NUM_TRADES,TOTAL_TRADED_VOLUME,ORDER_IMBALANCE,1MIN_RETURN,NEXT_1MIN_RETURN
0,C,2024-01-02,2024-01-02 09:30:00,0.036966,0.000721,0.000293,51.280,2.744828,3.200000,0.017096,3154,4669,41191,302,41191,-0.036780,NaN,0.001073
1,C,2024-01-02,2024-01-02 09:31:00,0.042653,0.000833,0.000367,51.335,2.224490,2.546939,0.063801,3333,4003,20013,248,20013,-0.033478,0.001073,0.002922
2,C,2024-01-02,2024-01-02 09:32:00,0.067746,0.001336,0.000924,51.485,2.689589,5.562560,0.044754,22172,22192,619970,1049,619970,-0.000032,0.002922,0.001845
3,C,2024-01-02,2024-01-02 09:33:00,0.080236,0.001581,0.002352,51.580,2.543906,2.503277,-0.041130,13154,8734,72906,763,72906,0.060626,0.001845,0.001648
4,C,2024-01-02,2024-01-02 09:34:00,0.087648,0.001741,0.008247,51.665,3.194444,2.557407,-0.093922,13499,3720,49435,540,49435,0.197815,0.001648,-0.002807


### Test Train Data 

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=24)

scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Linear Regression